# VGG

    '''
    参数设置参考Justin 598WI2022课件

    todo
    模型较大2070S很难训练，cifar10大约4个小时5个epoch
    1%|▏         | 11/782 [03:47<3:33:59, 16.65s/it]

    '''

<img src="./image/vgg.png" alt="Model Image" width="800">

In [1]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

d:\sgd-代码库\torch2.0-paly\sgd_deep_learning\sgd_cv


In [2]:
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

from sgd_cv.model import VGG16, VGG19

## 测试模型

In [3]:
# 测试模型结构
model = VGG16(num_classes=10)
print(model)

# 随机生成一个批次的输入 (cifar10 图像大小: 3x227x227)
input_tensor = torch.randn(1, 3, 227, 227) # 是否设为224
output = model(input_tensor)

print(f"输入张量大小: {input_tensor.shape}")
print(f"输出张量大小: {output.shape}")  # 应为 [1, 10]

VGG16(
  (features): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (2): ConvBlock(
      (block): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)

## 训练模型

In [4]:
# 数据集加载
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224), # 224
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225]),
                                ])
# CIFAR10
train_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = VGG16(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

Files already downloaded and verified
Files already downloaded and verified
cuda


In [5]:
# 训练
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

  2%|▏         | 14/782 [02:39<4:05:02, 19.14s/it]

In [ ]:
# 测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")